# Skyrmion Interactive — Tutorial and Live Viewer

This notebook is a tutorial-oriented merge of the `Skyrmion_Lattice.ipynb` setup and the live visualization widget from `LiveOne.ipynb`.
The goals are:
- Explain the physics background behind skyrmions and atomistic spin dynamics (ASD).
- Build a simple 2D skyrmion-lattice input using the UppASD Python API.
- Run the initialization phase to produce restart files.
- Launch an interactive, Jupyter-native live viewer to explore LLG / MC stepping.

Read each explanatory text cell before running the following code cells. The code cells are intentionally explicit to serve as a learning resource.

## Physics background

A *skyrmion* is a topological magnetic texture where the magnetization wraps the unit sphere. In thin magnetic films with chiral interactions (Dzyaloshinskii–Moriya interaction, DMI) and competing exchange, compact skyrmions can form and arrange in a lattice.

Atomistic spin dynamics (ASD) integrates the Landau–Lifshitz–Gilbert (LLG) equations (plus stochastic thermal fields for finite temperature) for each atomic magnetic moment. UppASD performs these integrations efficiently in Fortran and exposes a Python orchestration layer.

This tutorial demonstrates how to construct a minimal skyrmion-lattice model, run the initialization phase, and then interactively step dynamics while observing the spin configuration.

In [ ]:
# Imports and environment checks
import importlib.util
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Optional: install uppasd from wheels if not available (uncomment when needed)
if importlib.util.find_spec('uppasd') is None:
    print('Installing uppasd...')
    !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

# UppASD API imports used in this tutorial
from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults
from uppasd.run.live_simulator import LiveSimulator
from uppasd.run.notebook_viewer import NotebookLiveViewer, ViewerConfig

print('Imports OK')

## Build a minimal 2D unit cell

We construct a tiny unit cell and set a uniform initial magnetization along +z. The following cell is deliberately compact to focus on the ASD workflow rather than material realism.

In [ ]:
# Lattice and initial moments
a = 1.0  # lattice spacing (arbitrary units)
cell = np.diag([a, a, a])
positions = np.array([[0.0, 0.0, 0.0]])  # single atom per unit cell (expand using supercell later)
natom = positions.shape[0]
species = np.ones(natom, dtype=int)
moments = np.zeros((natom, 3))
moments[:, 2] = 1.0  # magnetization pointing along +z
system = SpinSystem(cell, positions, species, moments)

print('Unit cell created: natom=', natom)

## Define interactions (exchange + DMI)

The chiral DMI combined with ferromagnetic exchange stabilizes skyrmion textures under suitable field and temperature conditions. Below we add example nearest-neighbour bonds; for a real study, increase system size and carefully choose parameters.

In [ ]:
exchange = ExchangeShellTable()
dmi = DMIShellTable()

# Example parameters (tune for interesting textures)
J = 1.0
D = 0.81650

# Add a couple of bonds for the tiny cell (this minimal example is illustrative only)
exchange.add_bond(1, 1, 1, [1, 0, 0], J)
exchange.add_bond(1, 1, 1, [-1, 0, 0], J)
exchange.add_bond(1, 1, 1, [0, 1, 0], J)
exchange.add_bond(1, 1, 1, [0, -1, 0], J)
dmi.add_bond(1, 1, 1, [1, 0, 0], [D, 0.0, 0.0])
dmi.add_bond(1, 1, 1, [-1, 0, 0], [-D, 0.0, 0.0])
dmi.add_bond(1, 1, 1, [0, 1, 0], [0.0, D, 0.0])
dmi.add_bond(1, 1, 1, [0, -1, 0], [0.0, -D, 0.0])

print('Interactions added (example)')

## Assemble `inpsd.dat` (initialization and simulation parameters)

We create an `ASDInput` block with an initial-phase (ip) that seeds the LLG simulation. The values used here are intended as starting points for experimentation.

In [ ]:
inp = ASDInput()
Nx = 64
Ny = 64
inp.block('system').set(ncell=(Nx, Ny, 1), bc=('P','P','0'), do_prnstruct=2)
inp.block('initial').set(ip_mode='S', ip_nphase=(1, '\n', 100, 0.0, 1e-15, 1.0), ip_hfield=(0.0, 0.0, -150.0), initmag=1)
inp.block('simulation').set(mode='S', nstep=5000, timestep=1e-15, damping=0.5, temperature=5.0, hfield=(0.0, 0.0, -150.0))
inp.block('measurables').set(plotenergy=1, do_avrg='Y', do_cumu='Y', do_prn_beff='Y', skyno='T', do_skyno_den='Y', do_skyno_cmass='Y', do_tottraj='Y')

print('ASDInput prepared')

## Run initialization and save input files

The initialization phase populates the run directory with `posfile`, `momfile`, and other files. We create a workspace and save all input files there.

In [ ]:
workdir = 'skyrmion_interactive_run'
ws = ASDWorkspace(workdir, clean=True)

# Prepare workspace and write input/restart files required by LiveSimulator
ws.prepare(system=system, inp=inp, exchange=exchange, dmi=dmi)

# Note: We intentionally do not instantiate or run `UppASDSimulator` here.
# The interactive `LiveSimulator` only needs the prepared input files present in `workdir`.

print('Input files written to', workdir)

## Start the LiveSimulator and launch the Notebook live viewer

Now we create a `LiveSimulator` using the workspace we prepared and show the `NotebookLiveViewer`. The live viewer lets you press buttons to run short LLG/MC steps and observe the spin texture in the notebook output area.

In [ ]:
# The LiveSimulator expects a prepared workspace with restart files
workspace = ASDWorkspace(workdir, clean=False)
live = LiveSimulator(workspace)
live.initialize()

viewer = NotebookLiveViewer(live, ViewerConfig(scale=20, stride=1, figsize=(6,6)));
viewer.show();

# You can now interact with the widget (LLG step, MC, Heatbath, reset)

print('Live viewer running — use the controls to step dynamics');

## Exercise

- Experiment with `temperature` and `hfield` and observe stability of skyrmions.
- Increase the supercell (`Nx`, `Ny`) to build a larger skyrmion lattice. (re-run the initialization)
- Experiment with `exchange` and `dmi` and examine the phase diagram. (re-run the initialization)
